In [51]:
import geopandas as gpd
import pandas as pd
import numpy as np
from libpysal.weights import Queen
pd.read_csv('TX_Validation.csv')
data = pd.read_csv('TX_Validation.csv')

data['Deaths'] = pd.to_numeric(data['Deaths'], errors='coerce')
data['Crude Rate'] = pd.to_numeric(data['Crude Rate'], errors='coerce')
data['Population'] = pd.to_numeric(data['Population'], errors='coerce')

print(data.tail())

       County  County Code Ten-Year Age Groups  Deaths  Population  Crude Rate
3043  Zavala         48507         55-64 years     NaN     23604.0         NaN
3044  Zavala         48507         65-74 years    13.0     16967.0   76.619320
3045  Zavala         48507         75-84 years    14.0     10235.0  136.785540
3046  Zavala         48507           85+ years    22.0      3636.0  605.060506
3047  Zavala         48507          Not Stated     0.0         NaN    0.000000


In [52]:
data['Population'] = pd.to_numeric(data['Population'], errors='coerce')

In [53]:
import warnings
warnings.simplefilter("ignore")
import libpysal 
from libpysal.weights import Queen
import numpy as np
import geopandas
import pandas
%matplotlib inline
import matplotlib.pyplot as plt
from splot.libpysal import plot_spatial_weights


path = gdf = gpd.read_file('TexasCounties/County.shp')
df = gpd.read_file('TexasCounties/County.shp')

In [54]:
qw = Queen.from_dataframe(df)

In [55]:
qw

In [56]:
def identify_county_by_index(df, index):
    try:
        county_name = df.loc[index, "County_Name"]
        return county_name
    except KeyError:
        return f"County not found for index: {index}"

In [57]:

def replace_suppressed_crude_rates_and_calculate_deaths(df, data, crude_rate_mapping, qw, geo_id_col, mortality_id_col):
    df[geo_id_col] = df[geo_id_col].astype(str)
    data[mortality_id_col] = data[mortality_id_col].astype(str)
    data['Population'] = pd.to_numeric(data['Population'], errors='coerce')  # Convert Population to numeric
    
    merged_gdf = df.merge(data, left_on=geo_id_col, right_on=mortality_id_col)
    merged_gdf = gpd.GeoDataFrame(merged_gdf, geometry='geometry')

    for index, row in merged_gdf.iterrows():
        if pd.isna(row['Crude Rate']) and row['Ten-Year Age Groups'] != "Not Stated":
            valid_neighbor_found = False
            for neighbor_index in qw.neighbors.get(index, []):
                neighbor = merged_gdf.iloc[neighbor_index]
                if neighbor['Ten-Year Age Groups'] == row['Ten-Year Age Groups'] and not pd.isna(neighbor['Crude Rate']) and neighbor['Crude Rate'] > 0.0:
                    merged_gdf.at[index, 'Crude Rate'] = neighbor['Crude Rate']
                    valid_neighbor_found = True
                    break

            if not valid_neighbor_found:
                age_group = row['Ten-Year Age Groups']
                if age_group in crude_rate_mapping:
                    merged_gdf.at[index, 'Crude Rate'] = crude_rate_mapping[age_group]

    # Calculate 'Deaths' for rows with NaN or empty 'Deaths'
    for index, row in merged_gdf.iterrows():
        if pd.isna(row['Deaths']) and row['Ten-Year Age Groups'] != "Not Stated":
            if pd.notnull(row['Population']) and row['Population'] > 0 and pd.notnull(row['Crude Rate']):
                calculated_deaths = (row['Crude Rate'] * row['Population']) / 100000
                merged_gdf.at[index, 'Deaths'] = np.ceil(calculated_deaths)  # Ensures death count is rounded up

    # Recalculate 'Crude Rate' to ensure it matches the newly estimated death counts
    for index, row in merged_gdf.iterrows():
        if pd.notnull(row['Deaths']) and pd.notnull(row['Population']) and row['Population'] > 0:
            merged_gdf.at[index, 'Crude Rate'] = (row['Deaths'] / row['Population']) * 100000

    return merged_gdf



In [58]:
df.reset_index(drop=True, inplace=True)
qw = Queen.from_dataframe(df)
df = gpd.read_file('TexasCounties/County.shp')

islands = qw.islands
if islands:
    print("Islands found at indices:", islands)


crude_rate_mapping = {
    
    "< 1 year": 6.5,
    "1-4 years": 0.4,
    "5-14 years": 0.2,
    "15-24 years": 0.3,
    "25-34 years": 0.8,
    "35-44 years": 2,
    "45-54 years": 5.5,
    "55-64 years": 13.3,
    "65-74 years": 30.9,
    "75-84 years": 76.3,
    "85+ years": 182.8,
    "Not Stated" : 0.0
}

replace_suppressed_crude_rates_and_calculate_deaths(df, data, crude_rate_mapping, qw, 'CNTY_FIPS', 'County Code')
print(result_gdf.head())



   OBJECTID  GID  CMPTRL_NBR   CNTY_NM      DIST_NM  DPS_NBR CNTY_FIPS  \
0         1   29           7  Atascosa  San Antonio        7     48013   
1         1   29           7  Atascosa  San Antonio        7     48013   
2         1   29           7  Atascosa  San Antonio        7     48013   
3         1   29           7  Atascosa  San Antonio        7     48013   
4         1   29           7  Atascosa  San Antonio        7     48013   

   CNTY_NBR  DIST_NBR MSA1990  ...                                GlobalID  \
0         7        15       Y  ...  {5024B0A9-D5E0-4E6A-9655-10EA956E9CDF}   
1         7        15       Y  ...  {5024B0A9-D5E0-4E6A-9655-10EA956E9CDF}   
2         7        15       Y  ...  {5024B0A9-D5E0-4E6A-9655-10EA956E9CDF}   
3         7        15       Y  ...  {5024B0A9-D5E0-4E6A-9655-10EA956E9CDF}   
4         7        15       Y  ...  {5024B0A9-D5E0-4E6A-9655-10EA956E9CDF}   

  SHAPE_Leng SHAPE_Area                                           geometry  \
0   2.27

In [59]:
result_gdf = replace_suppressed_crude_rates_and_calculate_deaths(df, data, crude_rate_mapping, qw, 'CNTY_FIPS', 'County Code')
print(result_gdf.tail())


      OBJECTID  GID  CMPTRL_NBR  CNTY_NM   DIST_NM  DPS_NBR CNTY_FIPS  \
3043       254  249         103  Hartley  Amarillo      103     48205   
3044       254  249         103  Hartley  Amarillo      103     48205   
3045       254  249         103  Hartley  Amarillo      103     48205   
3046       254  249         103  Hartley  Amarillo      103     48205   
3047       254  249         103  Hartley  Amarillo      103     48205   

      CNTY_NBR  DIST_NBR MSA1990  ...                                GlobalID  \
3043       104         4       N  ...  {66052684-912F-4EDC-9C31-7AD5AD9CE9A5}   
3044       104         4       N  ...  {66052684-912F-4EDC-9C31-7AD5AD9CE9A5}   
3045       104         4       N  ...  {66052684-912F-4EDC-9C31-7AD5AD9CE9A5}   
3046       104         4       N  ...  {66052684-912F-4EDC-9C31-7AD5AD9CE9A5}   
3047       104         4       N  ...  {66052684-912F-4EDC-9C31-7AD5AD9CE9A5}   

     SHAPE_Leng SHAPE_Area                                           geome

In [60]:
# define the columns
columns_to_keep = ['County', 'County Code', 'Ten-Year Age Groups', 'Deaths', 'Population', 'Crude Rate']

mortality_results = result_gdf[columns_to_keep]

# Export the selected columns to a CSV file
mortality_results.to_csv('TX_validation_complete.csv', index=False)

In [61]:

pd.read_csv('TX_validation_complete.csv')

,County,County Code,Ten-Year Age Groups,Deaths,Population,Crude Rate
0,Atascosa,48013,< 1 year,0.0,14095.0,0.000000
1,Atascosa,48013,1-4 years,0.0,58269.0,0.000000
2,Atascosa,48013,5-14 years,1.0,154923.0,0.645482
3,Atascosa,48013,15-24 years,1.0,132904.0,0.752423
4,Atascosa,48013,25-34 years,0.0,117778.0,0.000000
...,...,...,...,...,...,...
3043,Hartley,48205,55-64 years,2.0,12887.0,15.519516
3044,Hartley,48205,65-74 years,0.0,8695.0,0.000000
3045,Hartley,48205,75-84 years,5.0,5412.0,92.387288
3046,Hartley,48205,85+ years,5.0,2484.0,201.288245


In [62]:
pd.read_csv('standard_pop.csv')

,State,State Code,Ten-Year Age Groups,Weight
0,Texas,48,< 1 year,0.014
1,Texas,48,1-4 years,0.055
2,Texas,48,5-14 years,0.146
3,Texas,48,15-24 years,0.139
4,Texas,48,25-34 years,0.136
5,Texas,48,35-44 years,0.163
6,Texas,48,45-54 years,0.135
7,Texas,48,55-64 years,0.087
8,Texas,48,65-74 years,0.066
9,Texas,48,75-84 years,0.045


In [64]:
import pandas as pd

def calculate_direct_aadr(mortality_csv, weights_csv, output_csv):
    # Load the datasets
    mortality_data = pd.read_csv(mortality_csv)
    weights_data = pd.read_csv(weights_csv)
    
    # Merge
    merged_data = pd.merge(mortality_data, weights_data[['Ten-Year Age Groups', 'Weight']], on='Ten-Year Age Groups')
    
    # Calculate Direct Rate for each row
    merged_data['Direct Rate'] = merged_data['Crude Rate'] * merged_data['Weight']
    
    # Aggregate Direct Rates by County
    direct_aadr_output = merged_data.groupby('County')['Direct Rate'].sum().reset_index()
    
    # Save the results to a new CSV file
    direct_aadr_output.to_csv(output_csv, index=False)

    return direct_aadr_output

mortality_csv = 'TX_Validation_complete.csv'
weights_csv = 'standard_pop.csv'
output_csv = 'Direct_AADR_Output.csv'
direct_aadr_output = calculate_direct_aadr(mortality_csv, weights_csv, output_csv)

print(direct_aadr_output)


        County  Direct Rate
0    Anderson     13.433073
1     Andrews     15.175867
2    Angelina     14.342684
3     Aransas     13.976597
4      Archer     10.950266
..         ...          ...
249      Wood     13.466113
250    Yoakum     11.607557
251     Young     11.494808
252    Zapata     22.198828
253    Zavala     23.759341

[254 rows x 2 columns]
